In [18]:
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [31]:
def load_asd_data_with_pandas(data_path: str) -> pd.DataFrame:
    """
    Загружает все parquet файлы из папки asd в один pandas DataFrame

    Args:
        data_path: путь к папке с данными

    Returns:
        pd.DataFrame: объединенный DataFrame со всеми данными
    """
    # Получаем все parquet файлы из папки
    parquet_files = glob.glob(os.path.join(data_path, "part-*.parquet"))

    if not parquet_files:
        raise ValueError(f"Не найдено parquet файлов в папке {data_path}")

    print(f"Найдено {len(parquet_files)} parquet файлов")

    # Загружаем все файлы в список DataFrame'ов
    dataframes = []
    for file_path in parquet_files:
        # print(f"Загружаем файл: {os.path.basename(file_path)}")
        df = pd.read_parquet(file_path)
        dataframes.append(df)

    # Объединяем все DataFrame'ы в один
    combined_df = pd.concat(dataframes, ignore_index=True)

    print(f"Общий размер данных: {combined_df.shape}")
    print(f"Колонки: {list(combined_df.columns)}")

    return combined_df

# Загружаем данные
agab_df = load_asd_data_with_pandas('./asd')

Найдено 20 parquet файлов
Общий размер данных: (1227083, 11)
Колонки: ['dataset', 'heavy_sequence', 'light_sequence', 'scfv', 'affinity_type', 'affinity', 'antigen_sequence', 'confidence', 'nanobody', 'metadata', 'processed_measurement']


### Cтатистика

In [ ]:
# Проверяем дубликаты
temp_df = agab_df.copy()
temp_df['metadata_json'] = temp_df['metadata'].apply(lambda x: json.dumps(x, sort_keys=True) if isinstance(x, dict) else str(x))
duplicates_mask = temp_df.drop(columns='metadata').duplicated()
duplicates_count = duplicates_mask.sum()

# Проверяем дубликаты, исключая столбец metadata (используем все столбцы кроме metadata)
duplicates_without_metadata_mask = agab_df.drop(columns='metadata').duplicated()
duplicates_without_metadata_count = duplicates_without_metadata_mask.sum()

# Проверяем дубликаты по heavy_sequence, light_sequence и antigen_sequence
duplicates_hla_mask = agab_df[['heavy_sequence', 'light_sequence', 'antigen_sequence']].duplicated()
duplicates_hla_count = duplicates_hla_mask.sum()

# Проверяем дубликаты по heavy_sequence и light_sequence
duplicates_hl_mask = agab_df[['heavy_sequence', 'light_sequence']].duplicated()
duplicates_hl_count = duplicates_hl_mask.sum()


print(f"Всего строк: {len(agab_df)}")

print(f"\nДубликатов (с учетом metadata): {duplicates_count}")
print(f"Уникальных строк (с учетом metadata): {len(agab_df) - duplicates_count}")

print(f"\nДубликатов (без metadata): {duplicates_without_metadata_count}")
print(f"Уникальных строк (без metadata): {len(agab_df) - duplicates_without_metadata_count}")

print(f"\nДубликатов по heavy_sequence, light_sequence и antigen_sequence: {duplicates_hla_count}")
print(f"Уникальных строк по heavy_sequence, light_sequence и antigen_sequence: {len(agab_df) - duplicates_hla_count}")

print(f"\nДубликатов по heavy_sequence и light_sequence: {duplicates_hl_count}")
print(f"Уникальных строк по heavy_sequence и light_sequence: {len(agab_df) - duplicates_hl_count}")


Всего строк: 1227083

Дубликатов (с учетом metadata): 28602
Уникальных строк (с учетом metadata): 1198481

Дубликатов (кроме metadata): 56436
Уникальных строк (кроме metadata): 1170647

Дубликатов по heavy_sequence, light_sequence и antigen_sequence: 129100
Уникальных строк по heavy_sequence, light_sequence и antigen_sequence: 1097983

Дубликатов по heavy_sequence и light_sequence: 361803
Уникальных строк по heavy_sequence и light_sequence: 865280


### По датасетам

In [ ]:
temp_df = agab_df.copy()
temp_df['is_duplicate'] = duplicates_mask

# Группируем по dataset и считаем статистику
dataset_duplicates = temp_df.drop(columns='metadata').groupby('dataset').agg({
    'is_duplicate': ['count', 'sum']
}).round(4)

dataset_duplicates.columns = ['total_rows', 'duplicates']
dataset_duplicates = dataset_duplicates.reset_index()
dataset_duplicates['duplicate_pct'] = (dataset_duplicates['duplicates'] / dataset_duplicates['total_rows'] * 100).round(1)
dataset_duplicates['unique_rows'] = dataset_duplicates['total_rows'] - dataset_duplicates['duplicates']

# Сортируем по количеству дубликатов
dataset_duplicates = dataset_duplicates.sort_values('duplicates', ascending=False)

print('дубль — одинаковы все колонки\n')
print("\n=== АНАЛИЗ ДУБЛИКАТОВ ПО ДАТАСЕТАМ ===\n")

print("Дубликаты по датасетам:")
print(dataset_duplicates.to_string(index=False))

дубль — одинаковы все колонки


=== АНАЛИЗ ДУБЛИКАТОВ ПО ДАТАСЕТАМ ===

Дубликаты по датасетам:
                dataset  total_rows  duplicates  duplicate_pct  unique_rows
                    hiv       48008       24715           51.5        23293
                 biomap        2725        1402           51.4         1323
  structures-antibodies        2711         999           36.8         1712
  structures-nanobodies        1258         566           45.0          692
                genbank        2989         480           16.1         2509
               covid-19       54625         303            0.6        54322
                   abbd      155853         132            0.1       155721
               alphaseq      198703           4            0.0       198699
flab_shanehsazzadeh2023         446           1            0.2          445
               skempiv2         434           0            0.0          434
                   rmna          10           0            0.0      

In [48]:
temp_df = agab_df.copy()
temp_df['is_duplicate'] = duplicates_without_metadata_mask

# Группируем по dataset и считаем статистику
dataset_duplicates = temp_df.drop(columns='metadata').groupby('dataset').agg({
    'is_duplicate': ['count', 'sum']
}).round(4)

dataset_duplicates.columns = ['total_rows', 'duplicates']
dataset_duplicates = dataset_duplicates.reset_index()
dataset_duplicates['duplicate_pct'] = (dataset_duplicates['duplicates'] / dataset_duplicates['total_rows'] * 100).round(1)
dataset_duplicates['unique_rows'] = dataset_duplicates['total_rows'] - dataset_duplicates['duplicates']

# Сортируем по количеству дубликатов
dataset_duplicates = dataset_duplicates.sort_values('duplicates', ascending=False)

print('дубль — одинаковы все колонки, кроме metadata\n')
print("\n=== АНАЛИЗ ДУБЛИКАТОВ ПО ДАТАСЕТАМ ===\n")

print("Дубликаты по датасетам:")
print(dataset_duplicates.to_string(index=False))

дубль — одинаковы все колонки, кроме metadata


=== АНАЛИЗ ДУБЛИКАТОВ ПО ДАТАСЕТАМ ===

Дубликаты по датасетам:
                dataset  total_rows  duplicates  duplicate_pct  unique_rows
               covid-19       54625       27482           50.3        27143
                    hiv       48008       24715           51.5        23293
                 biomap        2725        1402           51.4         1323
  structures-antibodies        2711        1170           43.2         1541
                genbank        2989         717           24.0         2272
  structures-nanobodies        1258         649           51.6          609
                patents      217463         164            0.1       217299
                   abbd      155853         132            0.1       155721
               alphaseq      198703           4            0.0       198699
flab_shanehsazzadeh2023         446           1            0.2          445
               skempiv2         434           0     

In [102]:
temp_df = agab_df.copy()
temp_df['is_duplicate'] = duplicates_hla_mask

# Группируем по dataset и считаем статистику
dataset_duplicates = temp_df.drop(columns='metadata').groupby('dataset').agg({
    'is_duplicate': ['count', 'sum']
}).round(4)

dataset_duplicates.columns = ['total_rows', 'duplicates']
dataset_duplicates = dataset_duplicates.reset_index()
dataset_duplicates['duplicate_pct'] = (dataset_duplicates['duplicates'] / dataset_duplicates['total_rows'] * 100).round(1)
dataset_duplicates['unique_rows'] = dataset_duplicates['total_rows'] - dataset_duplicates['duplicates']

# Сортируем по количеству дубликатов
dataset_duplicates = dataset_duplicates.sort_values('duplicates', ascending=False)

print('дубль — одинаковы heavy_sequence, light_sequence, antigen_sequence\n')
print("\n=== АНАЛИЗ ДУБЛИКАТОВ ПО ДАТАСЕТАМ ===\n")

print("Дубликаты по датасетам:")
print(dataset_duplicates.to_string(index=False))

дубль — одинаковы heavy_sequence, light_sequence, antigen_sequence


=== АНАЛИЗ ДУБЛИКАТОВ ПО ДАТАСЕТАМ ===

Дубликаты по датасетам:
                dataset  total_rows  duplicates  duplicate_pct  unique_rows
                   abbd      155853       66907           42.9        88946
               covid-19       54625       27761           50.8        26864
                    hiv       48008       24815           51.7        23193
               alphaseq      198703        4836            2.4       193867
                 biomap        2725        1639           60.1         1086
  structures-antibodies        2711        1174           43.3         1537
                genbank        2989         803           26.9         2186
  structures-nanobodies        1258         649           51.6          609
                patents      217463         208            0.1       217255
                ab-bind         283         124           43.8          159
             literature        

In [ ]:
temp_df = agab_df.copy()
temp_df['is_duplicate'] = duplicates_hl_mask

# Группируем по dataset и считаем статистику
dataset_duplicates = temp_df.drop(columns='metadata').groupby('dataset').agg({
    'is_duplicate': ['count', 'sum']
}).round(4)

dataset_duplicates.columns = ['total_rows', 'duplicates']
dataset_duplicates = dataset_duplicates.reset_index()
dataset_duplicates['duplicate_pct'] = (dataset_duplicates['duplicates'] / dataset_duplicates['total_rows'] * 100).round(1)
dataset_duplicates['unique_rows'] = dataset_duplicates['total_rows'] - dataset_duplicates['duplicates']

# Сортируем по количеству дубликатов
dataset_duplicates = dataset_duplicates.sort_values('duplicates', ascending=False)

print('дубль — одинаковы heavy_sequence и light_sequence\n')
print("\n=== АНАЛИЗ ДУБЛИКАТОВ ПО ДАТАСЕТАМ ===\n")

print("Дубликаты по датасетам:")
print(dataset_duplicates.to_string(index=False))

дубль — одинаковы heavy_sequence и light_sequence


=== АНАЛИЗ ДУБЛИКАТОВ ПО ДАТАСЕТАМ ===

Дубликаты по датасетам:
                dataset  total_rows  duplicates  duplicate_pct  unique_rows
                patents      217463      186336           85.7        31127
                   abbd      155853       66907           42.9        88946
               covid-19       54625       47867           87.6         6758
                    hiv       48008       47817           99.6          191
               alphaseq      198703        4836            2.4       193867
                genbank        2989        2428           81.2          561
                 biomap        2725        1997           73.3          728
  structures-antibodies        2711        1390           51.3         1321
             literature        5636         973           17.3         4663
  structures-nanobodies        1258         706           56.1          552
                   dlgo         360         324 

### По файлам


In [ ]:
print('дубль — одинаковы все колонки, кроме metadata\n')
print("=== АНАЛИЗ ДУБЛИКАТОВ ПО ФАЙЛАМ ===\n")

parquet_files = glob.glob(os.path.join('./asd', "part-*.parquet"))
parquet_files.sort()

file_duplicates = []
for file_path in parquet_files:
    df = pd.read_parquet(file_path)
    file_name = os.path.basename(file_path)
    duplicates_in_file = df.drop(columns=['metadata']).duplicated().sum()
    file_duplicates.append({
        'file': file_name,
        'rows': len(df),
        'duplicates': duplicates_in_file,
        'duplicate_pct': duplicates_in_file / len(df) * 100 if len(df) > 0 else 0
    })

file_duplicates_df = pd.DataFrame(file_duplicates)
print("Дубликаты внутри каждого файла:")
print(file_duplicates_df.to_string(index=False))
print(f"\nОбщее количество дубликатов внутри файлов: {file_duplicates_df['duplicates'].sum()}")


=== АНАЛИЗ ДУБЛИКАТОВ ПО ФАЙЛАМ ===

Дубликаты внутри каждого файла:
                                                               file  rows  duplicates  duplicate_pct
part-00000-3a065afd-b2fa-4875-a6e5-911e95e3f86c-c000.snappy.parquet 61329        2777       4.528037
part-00001-74fdee0d-8448-4b0a-921c-f1f0ef356cdf-c000.snappy.parquet 60919        2740       4.497776
part-00002-634ededa-5adf-4170-93ba-2dac2bd74705-c000.snappy.parquet 63736        2550       4.000879
part-00003-427bc79e-0e40-4a8c-a2be-d0fbe09f03c0-c000.snappy.parquet 61152        2542       4.156855
part-00004-eb3dc336-995c-48bd-840f-49d411a89b8e-c000.snappy.parquet 60125        2936       4.883160
part-00005-846a5164-9ca8-4438-af5f-07ad5348f327-c000.snappy.parquet 61104        2816       4.608536
part-00006-b818506c-2926-406c-936b-66da5c9acdbc-c000.snappy.parquet 62158        3105       4.995334
part-00007-7ab8e466-84f8-43e0-8874-9c1bbf210d4e-c000.snappy.parquet 60256        2778       4.610329
part-00008-397aa529-5c

In [16]:
print("\n=== АНАЛИЗ ПЕРЕКРЫТИЙ МЕЖДУ ФАЙЛАМИ ===\n")

from itertools import combinations

file_unique_sets = {os.path.basename(fp): set(pd.read_parquet(fp).drop(columns=['metadata']).apply(tuple, axis=1)) for fp in parquet_files}

overlaps = [{'file1': f1, 'file2': f2, 'overlap_count': overlap, 'file1_size': len(s1), 'file2_size': len(s2), 
             'overlap_pct_file1': overlap / len(s1) * 100 if len(s1) > 0 else 0, 'overlap_pct_file2': overlap / len(s2) * 100 if len(s2) > 0 else 0}
            for (f1, s1), (f2, s2) in combinations(file_unique_sets.items(), 2) if (overlap := len(s1 & s2)) > 0]

if overlaps:
    overlaps_df = pd.DataFrame(overlaps)
    print("Найдены перекрытия между файлами:")
    print(overlaps_df.to_string(index=False))
    print(f"\nВсего пар файлов с перекрытиями: {len(overlaps)}")
    print(f"Общее количество перекрывающихся записей: {overlaps_df['overlap_count'].sum()}")
else:
    print("Перекрытий между файлами не обнаружено.")



=== АНАЛИЗ ПЕРЕКРЫТИЙ МЕЖДУ ФАЙЛАМИ ===

Перекрытий между файлами не обнаружено.


### Различия metadata в дублях

In [ ]:
# на примере covid-19
covid_df = agab_df[
    (agab_df['dataset'] == 'covid-19')
    & (agab_df['scfv'] == False)
    & (agab_df['nanobody'] == False)
]

In [35]:
from collections import defaultdict, Counter


print("=== АНАЛИЗ РАЗЛИЧИЙ В METADATA МЕЖДУ ДУБЛИКАТАМИ В COVID-19 ===\n")

# Находим дубликаты в covid_df по всем колонкам кроме metadata
covid_duplicates_mask = covid_df.drop(columns=['metadata']).duplicated(keep=False)
covid_duplicates = covid_df[covid_duplicates_mask].copy()

# Создаем ключ для группировки
covid_duplicates['duplicate_key'] = covid_duplicates.drop(columns=['metadata']).apply(tuple, axis=1)

# Получаем уникальные группы дубликатов
unique_duplicate_groups = covid_duplicates['duplicate_key'].unique()

print(f"Всего групп дубликатов: {len(unique_duplicate_groups)}\n")

# Функция для рекурсивного сравнения словарей и поиска различий
def find_different_keys(dict1, dict2, path=""):
    """Находит все ключи, которые различаются между двумя словарями"""
    different_keys = set()
    
    if not isinstance(dict1, dict) or not isinstance(dict2, dict):
        if dict1 != dict2:
            return {path} if path else set()
        return set()
    
    all_keys = set(dict1.keys()) | set(dict2.keys())
    
    for key in all_keys:
        current_path = f"{path}.{key}" if path else key
        val1 = dict1.get(key)
        val2 = dict2.get(key)
        
        if key not in dict1:
            different_keys.add(current_path)
        elif key not in dict2:
            different_keys.add(current_path)
        elif isinstance(val1, dict) and isinstance(val2, dict):
            different_keys.update(find_different_keys(val1, val2, current_path))
        elif val1 != val2:
            different_keys.add(current_path)
    
    return different_keys

# Собираем статистику по различиям
field_differences = Counter()
groups_with_differences = 0
groups_with_same_metadata = 0
total_groups_analyzed = 0

# Анализируем каждую группу дубликатов
for group_key in unique_duplicate_groups:
    group_data = covid_duplicates[covid_duplicates['duplicate_key'] == group_key]
    
    if len(group_data) < 2:
        continue
    
    total_groups_analyzed += 1
    
    # Сравниваем metadata между всеми парами в группе
    metadata_list = group_data['metadata'].tolist()
    
    # Проверяем, все ли metadata одинаковые
    all_same = True
    group_differences = set()
    
    for i in range(len(metadata_list)):
        for j in range(i + 1, len(metadata_list)):
            meta1 = metadata_list[i] if isinstance(metadata_list[i], dict) else {}
            meta2 = metadata_list[j] if isinstance(metadata_list[j], dict) else {}
            
            diff_keys = find_different_keys(meta1, meta2)
            if diff_keys:
                all_same = False
                group_differences.update(diff_keys)
    
    if all_same:
        groups_with_same_metadata += 1
    else:
        groups_with_differences += 1
        for key in group_differences:
            field_differences[key] += 1

print(f"Проанализировано групп: {total_groups_analyzed}")
print(f"Групп с одинаковым metadata: {groups_with_same_metadata} ({groups_with_same_metadata/total_groups_analyzed*100:.2f}%)")
print(f"Групп с различиями в metadata: {groups_with_differences} ({groups_with_differences/total_groups_analyzed*100:.2f}%)\n")

if field_differences:
    print("=== ПОЛЯ METADATA, КОТОРЫЕ ЧАЩЕ ВСЕГО РАЗЛИЧАЮТСЯ ===\n")
    for field, count in field_differences:
        pct = count / groups_with_differences * 100
        print(f"{field}: {count} групп ({pct:.2f}% от групп с различиями)")
    
    print(f"\nВсего уникальных полей с различиями: {len(field_differences)}")
else:
    print("Различий в metadata между дубликатами не найдено.")

# Показываем примеры групп с различиями
print("\n=== ПРИМЕРЫ ГРУПП С РАЗЛИЧИЯМИ В METADATA ===\n")

examples_shown = 0
for group_key in unique_duplicate_groups:
    if examples_shown >= 5:
        break
    
    group_data = covid_duplicates[covid_duplicates['duplicate_key'] == group_key].copy()
    
    if len(group_data) < 2:
        continue
    
    metadata_list = group_data['metadata'].tolist()
    
    # Проверяем, есть ли различия
    has_differences = False
    all_differences = set()
    
    for i in range(len(metadata_list)):
        for j in range(i + 1, len(metadata_list)):
            meta1 = metadata_list[i] if isinstance(metadata_list[i], dict) else {}
            meta2 = metadata_list[j] if isinstance(metadata_list[j], dict) else {}
            
            diff_keys = find_different_keys(meta1, meta2)
            if diff_keys:
                has_differences = True
                all_differences.update(diff_keys)
    
    if has_differences:
        examples_shown += 1
        print(f"{'='*80}")
        print(f"Пример {examples_shown}: Группа из {len(group_data)} дубликатов")
        print(f"Различающиеся поля: {', '.join(sorted(all_differences))}")
        print(f"{'='*80}\n")
        
        # Показываем первые 2 записи из группы
        for idx, (row_idx, row) in enumerate(group_data.head(2).iterrows()):
            print(f"--- Запись {idx+1} (индекс {row_idx}) ---")
            metadata_str = json.dumps(row['metadata'], indent=2, ensure_ascii=False) if isinstance(row['metadata'], dict) else str(row['metadata'])
            print(f"metadata:\n{metadata_str}\n")
        
        print()

# Удаляем временную колонку
covid_duplicates.drop(columns=['duplicate_key'], inplace=True)


=== АНАЛИЗ РАЗЛИЧИЙ В METADATA МЕЖДУ ДУБЛИКАТАМИ В COVID-19 ===

Всего групп дубликатов: 27142

Проанализировано групп: 27142
Групп с одинаковым metadata: 0 (0.00%)
Групп с различиями в metadata: 27142 (100.00%)

=== ПОЛЯ METADATA, КОТОРЫЕ ЧАЩЕ ВСЕГО РАЗЛИЧАЮТСЯ ===



ValueError: too many values to unpack (expected 2)

In [37]:
# они друг за другом идут в covid-19
for idx, meta in covid_df.iloc[1000:1002]['metadata'].items():
    print(f'Индекс {idx}:\n', json.dumps(meta, ensure_ascii=False, indent=2), '\n')

Индекс 55265:
 {
  "target_name": null,
  "target_pdb": "",
  "target_uniprot": "",
  "source_url": "https://academic.oup.com/bib/article/26/1/bbaf008/7964432",
  "heavy_riot_numbering": {
    "cdr1_aa": "GFTFTSSA",
    "cdr2_aa": "IVVGSGNT",
    "cdr3_aa": "AAPYCSGGSCFDGFDI",
    "sequence_alignment_aa": "QMQLVQSGPEVKKPGTSVKVSCKASGFTFTSSAVQWVRQARGQRLEWIGWIVVGSGNTNYAQKFQERVTITRDMSTSTAYMELSSLRSEDTAVYYCAAPYCSGGSCFDGFDIWGQGTMVTVSS"
  },
  "light_riot_numbering": {
    "cdr1_aa": null,
    "cdr2_aa": null,
    "cdr3_aa": null,
    "sequence_alignment_aa": null
  }
} 

Индекс 55266:
 {
  "target_name": "sars-cov2_omicron",
  "target_pdb": "",
  "target_uniprot": "",
  "source_url": "https://academic.oup.com/bioinformatics/article/37/5/734/5893556",
  "heavy_riot_numbering": {
    "cdr1_aa": "GFTFTSSA",
    "cdr2_aa": "IVVGSGNT",
    "cdr3_aa": "AAPYCSGGSCFDGFDI",
    "sequence_alignment_aa": "QMQLVQSGPEVKKPGTSVKVSCKASGFTFTSSAVQWVRQARGQRLEWIGWIVVGSGNTNYAQKFQERVTITRDMSTSTAYMELSSLRSEDTAVYYCAAP

In [34]:
print("=== АНАЛИЗ РАЗЛИЧИЙ В METADATA МЕЖДУ ДУБЛИКАТАМИ В AGAB ===\n")

# Находим дубликаты в agab_df по всем колонкам кроме metadata
agab_duplicates_mask = agab_df.drop(columns=['metadata']).duplicated(keep=False)
agab_duplicates = agab_df[agab_duplicates_mask].copy()

# Создаем ключ для группировки
agab_duplicates['duplicate_key'] = agab_duplicates.drop(columns=['metadata']).apply(tuple, axis=1)

# Получаем уникальные группы дубликатов
unique_duplicate_groups = agab_duplicates['duplicate_key'].unique()

print(f"Всего групп дубликатов: {len(unique_duplicate_groups)}\n")

# Функция для рекурсивного сравнения словарей и поиска различий
def find_different_keys(dict1, dict2, path=""):
    """Находит все ключи, которые различаются между двумя словарями"""
    different_keys = set()
    
    if not isinstance(dict1, dict) or not isinstance(dict2, dict):
        if dict1 != dict2:
            return {path} if path else set()
        return set()
    
    all_keys = set(dict1.keys()) | set(dict2.keys())
    
    for key in all_keys:
        current_path = f"{path}.{key}" if path else key
        val1 = dict1.get(key)
        val2 = dict2.get(key)
        
        if key not in dict1:
            different_keys.add(current_path)
        elif key not in dict2:
            different_keys.add(current_path)
        elif isinstance(val1, dict) and isinstance(val2, dict):
            different_keys.update(find_different_keys(val1, val2, current_path))
        elif val1 != val2:
            different_keys.add(current_path)
    
    return different_keys

# Собираем статистику по различиям
field_differences = Counter()
groups_with_differences = 0
groups_with_same_metadata = 0
total_groups_analyzed = 0

# Анализируем каждую группу дубликатов
for group_key in unique_duplicate_groups:
    group_data = agab_duplicates[agab_duplicates['duplicate_key'] == group_key]
    
    if len(group_data) < 2:
        continue
    
    total_groups_analyzed += 1
    
    # Сравниваем metadata между всеми парами в группе
    metadata_list = group_data['metadata'].tolist()
    
    # Проверяем, все ли metadata одинаковые
    all_same = True
    group_differences = set()
    
    for i in range(len(metadata_list)):
        for j in range(i + 1, len(metadata_list)):
            meta1 = metadata_list[i] if isinstance(metadata_list[i], dict) else {}
            meta2 = metadata_list[j] if isinstance(metadata_list[j], dict) else {}
            
            diff_keys = find_different_keys(meta1, meta2)
            if diff_keys:
                all_same = False
                group_differences.update(diff_keys)
    
    if all_same:
        groups_with_same_metadata += 1
    else:
        groups_with_differences += 1
        for key in group_differences:
            field_differences[key] += 1

print(f"Проанализировано групп: {total_groups_analyzed}")
print(f"Групп с одинаковым metadata: {groups_with_same_metadata} ({groups_with_same_metadata/total_groups_analyzed*100:.2f}%)")
print(f"Групп с различиями в metadata: {groups_with_differences} ({groups_with_differences/total_groups_analyzed*100:.2f}%)\n")

if field_differences:
    print("=== ПОЛЯ METADATA, КОТОРЫЕ ЧАЩЕ ВСЕГО РАЗЛИЧАЮТСЯ ===\n")
    for field, count in field_differences:
        pct = count / groups_with_differences * 100
        print(f"{field}: {count} групп ({pct:.2f}% от групп с различиями)")
    
    print(f"\nВсего уникальных полей с различиями: {len(field_differences)}")
else:
    print("Различий в metadata между дубликатами не найдено.")

# Показываем примеры групп с различиями
print("\n=== ПРИМЕРЫ ГРУПП С РАЗЛИЧИЯМИ В METADATA ===\n")

examples_shown = 0
for group_key in unique_duplicate_groups:
    if examples_shown >= 5:
        break
    
    group_data = agab_duplicates[agab_duplicates['duplicate_key'] == group_key].copy()
    
    if len(group_data) < 2:
        continue
    
    metadata_list = group_data['metadata'].tolist()
    
    # Проверяем, есть ли различия
    has_differences = False
    all_differences = set()
    
    for i in range(len(metadata_list)):
        for j in range(i + 1, len(metadata_list)):
            meta1 = metadata_list[i] if isinstance(metadata_list[i], dict) else {}
            meta2 = metadata_list[j] if isinstance(metadata_list[j], dict) else {}
            
            diff_keys = find_different_keys(meta1, meta2)
            if diff_keys:
                has_differences = True
                all_differences.update(diff_keys)
    
    if has_differences:
        examples_shown += 1
        print(f"{'='*80}")
        print(f"Пример {examples_shown}: Группа из {len(group_data)} дубликатов")
        print(f"Различающиеся поля: {', '.join(sorted(all_differences))}")
        print(f"{'='*80}\n")
        
        # Показываем первые 2 записи из группы
        for idx, (row_idx, row) in enumerate(group_data.head(2).iterrows()):
            print(f"--- Запись {idx+1} (индекс {row_idx}) ---")
            
            # Показываем все колонки кроме duplicate_key
            display_cols = [col for col in group_data.columns if col not in ['duplicate_key']]
            
            for col in display_cols:
                if col == 'metadata':
                    # Красиво выводим metadata
                    metadata_str = json.dumps(row[col], indent=2, ensure_ascii=False) if isinstance(row[col], dict) else str(row[col])
                    print(f"{col}:\n{metadata_str}")
                else:
                    print(f"{col}: {row[col]}")
            
            print()
        
        print()

# Удаляем временную колонку
agab_duplicates.drop(columns=['duplicate_key'], inplace=True)


=== АНАЛИЗ РАЗЛИЧИЙ В METADATA МЕЖДУ ДУБЛИКАТАМИ В AGAB ===

Всего групп дубликатов: 53478

Проанализировано групп: 53478
Групп с одинаковым metadata: 25872 (48.38%)
Групп с различиями в metadata: 27606 (51.62%)

=== ТОП-20 ПОЛЕЙ METADATA, КОТОРЫЕ ЧАЩЕ ВСЕГО РАЗЛИЧАЮТСЯ ===

target_name: 27433 групп (99.37% от групп с различиями)
source_url: 27142 групп (98.32% от групп с различиями)
target_uniprot: 225 групп (0.82% от групп с различиями)
target_pdb: 133 групп (0.48% от групп с различиями)

Всего уникальных полей с различиями: 4

=== ПРИМЕРЫ ГРУПП С РАЗЛИЧИЯМИ В METADATA ===

Пример 1: Группа из 4 дубликатов
Различающиеся поля: target_pdb

--- Запись 1 (индекс 33) ---
dataset: structures-nanobodies
heavy_sequence: EVQLLESGGGLVQPGGSLRLSCAASGFRFDAEDMGWVRQAPGKGLEWVSSIYGPSGSTYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAKYTSPPQNHGFDYWGQGTLVTVSS
light_sequence: 
scfv: False
affinity_type: bool
affinity: 1.0
antigen_sequence: KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRNTDGSTDYGILQINSRWWCNDGRTPGSR

### В чем отличие когда одинаковые heavy_sequence и light_sequence?

In [83]:
print("=== АНАЛИЗ РАЗЛИЧИЙ В ПОЛЯХ ДЛЯ ДУБЛИКАТОВ ПО ПОСЛЕДОВАТЕЛЬНОСТЯМ (agab_df) ===\n")

# Поля для проверки
fields = [
    "dataset",
    "scfv",
    "affinity_type",
    "affinity",
    "antigen_sequence",
    "confidence",
    "nanobody",
    "processed_measurement",
]

# --- Поиск групп дубликатов по heavy_sequence + light_sequence ---
# Находим группы, где хотя бы 2 записи с одинаковой парой последовательностей
group_cols = ["heavy_sequence", "light_sequence"]
grouped = agab_df.groupby(group_cols, sort=False)

# Маска строк, которые входят в группы размером > 1
dup_mask = grouped[group_cols[0]].transform("size") > 1
hl_duplicates = agab_df.loc[dup_mask].copy()

# Переиспользуем groupby только на дубликатах
dup_groups = hl_duplicates.groupby(group_cols, sort=False)
unique_groups_count = dup_groups.ngroups

print(f"Всего групп дубликатов: {unique_groups_count}\n")

# --- Анализ различий по полям ---

# Для каждой группы считаем, сколько уникальных значений в каждом поле
# dropna=False, чтобы различия NaN / не-NaN тоже учитывались
nunique_per_group = dup_groups[fields].nunique(dropna=False)

# Булева матрица: True, если в группе по полю есть различия
diff_mask = nunique_per_group > 1

# Есть ли вообще различия в группе
group_has_diffs = diff_mask.any(axis=1)

groups_with_diffs = int(group_has_diffs.sum())
groups_identical = int((~group_has_diffs).sum())
total = groups_with_diffs + groups_identical if (groups_with_diffs + groups_identical) > 0 else 1

print(f"Групп с различиями: {groups_with_diffs} ({groups_with_diffs / total * 100:.2f}%)")
print(f"Групп без различий: {groups_identical} ({groups_identical / total * 100:.2f}%)\n")

# Количество строк (записей) с различиями и без различий
rows_with_diffs = group_has_diffs[group_has_diffs].index  # индексы групп с различиями
rows_without_diffs = group_has_diffs[~group_has_diffs].index  # индексы групп без различий

num_rows_with_diffs = dup_groups.size().loc[rows_with_diffs].sum() if len(rows_with_diffs) > 0 else 0
num_rows_without_diffs = dup_groups.size().loc[rows_without_diffs].sum() if len(rows_without_diffs) > 0 else 0
rows_total = num_rows_with_diffs + num_rows_without_diffs if (num_rows_with_diffs + num_rows_without_diffs) > 0 else 1

print(f"Строк (записей) с различиями: {num_rows_with_diffs} ({num_rows_with_diffs / rows_total * 100:.2f}%)")
print(f"Строк (записей) без различий: {num_rows_without_diffs} ({num_rows_without_diffs / rows_total * 100:.2f}%)\n")

# Считаем, по скольким группам отличается каждое поле
field_diffs_counts = diff_mask.sum().sort_values(ascending=False)

if groups_with_diffs > 0 and not field_diffs_counts.empty:
    print("=== ПОЛЯ, КОТОРЫЕ ЧАЩЕ ВСЕГО РАЗЛИЧАЮТСЯ ===\n")
    for field, count in field_diffs_counts.items():
        if count == 0:
            continue
        print(f"{field}: {count} групп ({count / groups_with_diffs * 100:.2f}%)")

=== АНАЛИЗ РАЗЛИЧИЙ В ПОЛЯХ ДЛЯ ДУБЛИКАТОВ ПО ПОСЛЕДОВАТЕЛЬНОСТЯМ (agab_df) ===

Всего групп дубликатов: 81583

Групп с различиями: 77385 (94.85%)
Групп без различий: 4198 (5.15%)

Строк (записей) с различиями: 263389 (96.65%)
Строк (записей) без различий: 9137 (3.35%)

=== ПОЛЯ, КОТОРЫЕ ЧАЩЕ ВСЕГО РАЗЛИЧАЮТСЯ ===

affinity: 72196 групп (93.29%)
antigen_sequence: 7308 групп (9.44%)
processed_measurement: 2149 групп (2.78%)
dataset: 217 групп (0.28%)
nanobody: 40 групп (0.05%)
confidence: 32 групп (0.04%)
affinity_type: 11 групп (0.01%)
scfv: 9 групп (0.01%)


In [109]:
paired_agab_df = agab_df[
    agab_df['light_sequence'].notna()
    & agab_df['heavy_sequence'].notna()
    & (agab_df['light_sequence'] != '')
    & (agab_df['heavy_sequence'] != '')
    & (agab_df['nanobody'] == False)
    # & (agab_df['scfv'] == False) # строка не меняет кол-ва записей
].copy()

print(len(paired_agab_df))

paired_agab_df.drop(columns=['metadata'], inplace=True)
paired_agab_df.drop_duplicates(inplace=True)

print(len(paired_agab_df))

713227
711908


In [ ]:
# код для объединения metadata
paired_agab_df = agab_df[
    agab_df['light_sequence'].notna()
    & agab_df['heavy_sequence'].notna()
    & (agab_df['light_sequence'] != '')
    & (agab_df['heavy_sequence'] != '')
    & (agab_df['nanobody'] == False)
    # & (agab_df['scfv'] == False) # строка не меняет кол-ва записей
].copy()


print(f"Исходное количество записей: {len(paired_agab_df)}")

# Группируем по всем столбцам, кроме 'metadata'
cols_except_meta = [col for col in paired_agab_df.columns if col != "metadata"]

# Для каждой группы собираем все значения metadata в список
grouped = (
    paired_agab_df
    .groupby(cols_except_meta, sort=False)["metadata"]
    .agg(list)  # из нескольких строк делаем список metadata
    .reset_index()
)

# Превращаем списки длины 1 обратно в одиночное значение,
# чтобы поведение осталось как в исходном коде
grouped["metadata"] = grouped["metadata"].apply(
    lambda lst: lst[0] if len(lst) == 1 else lst
)

paired_agab_df = grouped

print("\nПримеры записей с объединённым metadata:")
for idx, row in paired_agab_df.head(20).iterrows():
    if isinstance(row["metadata"], list) and len(row["metadata"]) > 1:
        print(f"\nЗапись {idx}: {len(row['metadata'])} объединённых metadata")
        print(f"  dataset: {row['dataset']}")
        print(f"  heavy_sequence: {str(row['heavy_sequence'])[:50]}...")
        print(f"  Количество metadata: {len(row['metadata'])}")
        print(row['metadata'])

print(f"\nПосле очистки дублей: {len(paired_agab_df)}")

In [85]:
print("=== АНАЛИЗ РАЗЛИЧИЙ В ПОЛЯХ ДЛЯ ДУБЛИКАТОВ ПО ПОСЛЕДОВАТЕЛЬНОСТЯМ (paired_agab_df) ===\n")

# Поля для проверки
fields = [
    "dataset",
    "scfv",
    "affinity_type",
    "affinity",
    "antigen_sequence",
    "confidence",
    "nanobody",
    "processed_measurement",
]

# --- Поиск групп дубликатов по heavy_sequence + light_sequence ---
# Находим группы, где хотя бы 2 записи с одинаковой парой последовательностей
group_cols = ["heavy_sequence", "light_sequence"]
grouped = paired_agab_df.groupby(group_cols, sort=False)

# Маска строк, которые входят в группы размером > 1
dup_mask = grouped[group_cols[0]].transform("size") > 1
hl_duplicates = paired_agab_df.loc[dup_mask].copy()

# Переиспользуем groupby только на дубликатах
dup_groups = hl_duplicates.groupby(group_cols, sort=False)
unique_groups_count = dup_groups.ngroups

print(f"Всего групп дубликатов: {unique_groups_count}\n")

# --- Анализ различий по полям ---

# Для каждой группы считаем, сколько уникальных значений в каждом поле
# dropna=False, чтобы различия NaN / не-NaN тоже учитывались
nunique_per_group = dup_groups[fields].nunique(dropna=False)

# Булева матрица: True, если в группе по полю есть различия
diff_mask = nunique_per_group > 1

# Есть ли вообще различия в группе
group_has_diffs = diff_mask.any(axis=1)

groups_with_diffs = int(group_has_diffs.sum())
groups_identical = int((~group_has_diffs).sum())
total = groups_with_diffs + groups_identical if (groups_with_diffs + groups_identical) > 0 else 1

print(f"Групп с различиями: {groups_with_diffs} ({groups_with_diffs / total * 100:.2f}%)")
print(f"Групп без различий: {groups_identical} ({groups_identical / total * 100:.2f}%)\n")

# Количество строк (записей) с различиями и без различий
rows_with_diffs = group_has_diffs[group_has_diffs].index  # индексы групп с различиями
rows_without_diffs = group_has_diffs[~group_has_diffs].index  # индексы групп без различий

num_rows_with_diffs = dup_groups.size().loc[rows_with_diffs].sum() if len(rows_with_diffs) > 0 else 0
num_rows_without_diffs = dup_groups.size().loc[rows_without_diffs].sum() if len(rows_without_diffs) > 0 else 0
rows_total = num_rows_with_diffs + num_rows_without_diffs if (num_rows_with_diffs + num_rows_without_diffs) > 0 else 1

print(f"Строк (записей) с различиями: {num_rows_with_diffs} ({num_rows_with_diffs / rows_total * 100:.2f}%)")
print(f"Строк (записей) без различий: {num_rows_without_diffs} ({num_rows_without_diffs / rows_total * 100:.2f}%)\n")

# Считаем, по скольким группам отличается каждое поле
field_diffs_counts = diff_mask.sum().sort_values(ascending=False)

if groups_with_diffs > 0 and not field_diffs_counts.empty:
    print("=== ПОЛЯ, КОТОРЫЕ ЧАЩЕ ВСЕГО РАЗЛИЧАЮТСЯ ===\n")
    for field, count in field_diffs_counts.items():
        if count == 0:
            continue
        print(f"{field}: {count} групп ({count / groups_with_diffs * 100:.2f}%)")

=== АНАЛИЗ РАЗЛИЧИЙ В ПОЛЯХ ДЛЯ ДУБЛИКАТОВ ПО ПОСЛЕДОВАТЕЛЬНОСТЯМ (paired_agab_df) ===

Всего групп дубликатов: 69486

Групп с различиями: 69486 (100.00%)
Групп без различий: 0 (0.00%)

Строк (записей) с различиями: 155375 (100.00%)
Строк (записей) без различий: 0 (0.00%)

=== ПОЛЯ, КОТОРЫЕ ЧАЩЕ ВСЕГО РАЗЛИЧАЮТСЯ ===

affinity: 66971 групп (96.38%)
antigen_sequence: 2538 групп (3.65%)
dataset: 174 групп (0.25%)
processed_measurement: 45 групп (0.06%)
affinity_type: 10 групп (0.01%)
confidence: 2 групп (0.00%)


In [115]:
# --- Вывод примеров групп с различиями ---

def short_seq(seq, max_len=50):
    """Аккуратно обрезает последовательность для вывода."""
    if pd.isna(seq):
        return seq
    s = str(seq)
    return s[:max_len] + "..." if len(s) > max_len else s

print("\n=== ПРИМЕРЫ ГРУПП С РАЗЛИЧИЯМИ ===\n")

shown = 0
max_examples = 100

for (h_seq, l_seq), group in dup_groups:
    if shown >= max_examples:
        break

    # Определяем различающиеся поля для этой группы
    diff_fields = [f for f in fields if group[f].nunique(dropna=False) > 1]
    if not diff_fields:
        continue

    shown += 1
    print(
        f"{'=' * 80}\n"
        f"Пример {shown}: Группа из {len(group)} записей\n"
        f"Различающиеся поля: {', '.join(diff_fields)}\n"
        f"{'=' * 80}\n"
    )

    # Показываем первые 2 записи в группе
    for idx, (row_idx, row) in enumerate(group.head().iterrows(), start=1):
        print(f"--- Запись {idx} (индекс {row_idx}) ---")
        for field in ["dataset"] + diff_fields:
            print(f"{field}: {row[field]}")
        print(f"heavy_sequence: {short_seq(row['heavy_sequence'])}")
        print(f"light_sequence:  {short_seq(row['light_sequence'])}\n")
        print(f"confidence:  {(row['confidence'])}\n")
        print(f"processed_measurement:  {(row['processed_measurement'])}\n")



=== ПРИМЕРЫ ГРУПП С РАЗЛИЧИЯМИ ===

Пример 1: Группа из 4 записей
Различающиеся поля: dataset, affinity_type, affinity, processed_measurement

--- Запись 1 (индекс 1) ---
dataset: ab-bind
dataset: ab-bind
affinity_type: ddg
affinity: 1.1
processed_measurement: 0.7800730860179147
heavy_sequence: QVQLQESGPGLVAPSQSLSITCTVSGFSLTGAGVNWVRQPPGKGLEWLGM...
light_sequence:  DIVLTQSPASLSASVGETVTITCRASGNIHNYLAWYQQKQGKSPQLLVYY...

confidence:  very_high

processed_measurement:  0.7800730860179147

--- Запись 2 (индекс 2) ---
dataset: skempiv2
dataset: skempiv2
affinity_type: ddg
affinity: 0.4601461720358293
processed_measurement: 0.7800730860179147
heavy_sequence: QVQLQESGPGLVAPSQSLSITCTVSGFSLTGAGVNWVRQPPGKGLEWLGM...
light_sequence:  DIVLTQSPASLSASVGETVTITCRASGNIHNYLAWYQQKQGKSPQLLVYY...

confidence:  very_high

processed_measurement:  0.7800730860179147

--- Запись 3 (индекс 735093) ---
dataset: skempiv2
dataset: skempiv2
affinity_type: bool
affinity: 1.0
processed_measurement: 1.0
heavy_sequence:

In [108]:
# Определим группы, в которых все поля совпадают кроме 'affinity'
# Список полей (кроме 'affinity'), которые будем сравнивать
fields_except_antigen = [col for col in paired_agab_df.columns if col not in ["affinity", "metadata"]]

# Группируем по этим полям, считаем размер группы
grouped = paired_agab_df.groupby(fields_except_antigen)

# Оставляем только те группы, где есть различия в 'affinity'
groups_with_affinity_diffs = []
for group_keys, group in grouped:
    if group["affinity"].nunique(dropna=False) > 1:
        groups_with_antigen_diffs.append(group)

# Собираем все записи из этих групп в единый датафрейм
if groups_with_antigen_diffs:
    antigen_diff_df = pd.concat(groups_with_antigen_diffs, ignore_index=True)
else:
    antigen_diff_df = pd.DataFrame(columns=paired_agab_df.columns)

# Строим распределение по датасетам
print("\n=== РАСПРЕДЕЛЕНИЕ ПО ДАТАCЕТАМ ДЛЯ ЗАПИСЕЙ, ГДЕ ВСЁ СОВПАДАЕТ, КРОМЕ AFFINITY ===\n")
print(antigen_diff_df['dataset'].value_counts())



=== РАСПРЕДЕЛЕНИЕ ПО ДАТАCЕТАМ ДЛЯ ЗАПИСЕЙ, ГДЕ ВСЁ СОВПАДАЕТ, КРОМЕ AFFINITY ===

dataset
abbd                     133550
patents                   20715
structures-antibodies       326
dlgo                         80
skempiv2                     62
genbank                      19
ab-bind                       4
Name: count, dtype: int64


In [106]:
# Определим группы, в которых все поля совпадают кроме 'Antigen Sequence'
# Список полей (кроме 'Antigen Sequence'), которые будем сравнивать
fields_except_antigen = [col for col in paired_agab_df.columns if col not in ["antigen_sequence", "metadata"]]

# Группируем по этим полям, считаем размер группы
grouped = paired_agab_df.groupby(fields_except_antigen)

# Оставляем только те группы, где есть различия в 'antigen_sequence'
groups_with_antigen_diffs = []
for group_keys, group in grouped:
    if group["antigen_sequence"].nunique(dropna=False) > 1:
        groups_with_antigen_diffs.append(group)

# Собираем все записи из этих групп в единый датафрейм
if groups_with_antigen_diffs:
    antigen_diff_df = pd.concat(groups_with_antigen_diffs, ignore_index=True)
else:
    antigen_diff_df = pd.DataFrame(columns=paired_agab_df.columns)

# Строим распределение по датасетам
print("\n=== РАСПРЕДЕЛЕНИЕ ПО ДАТАCЕТАМ ДЛЯ ЗАПИСЕЙ, ГДЕ ВСЁ СОВПАДАЕТ, КРОМЕ ANTIGEN SEQUENCE ===\n")
print(antigen_diff_df['dataset'].value_counts())



=== РАСПРЕДЕЛЕНИЕ ПО ДАТАCЕТАМ ДЛЯ ЗАПИСЕЙ, ГДЕ ВСЁ СОВПАДАЕТ, КРОМЕ ANTIGEN SEQUENCE ===

dataset
patents                  20715
structures-antibodies      326
dlgo                        80
genbank                     19
Name: count, dtype: int64
